In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
#hide
from nbdev.showdoc import show_doc
from getpass import getuser, getpass
from fiscaliza.redmine import detalhar_inspecao, validar_dicionario, relatar_inspecao
from rich.console import Console
console = Console()

# Fiscaliza

> Scripts para baixar informações sobre inspeções no Sistema Fiscaliza da Anatel e atualizar inspeções por meio da API do Redmine.

## Instalação

`pip install fiscaliza`

## Como utilizar

O `Fiscaliza` da Anatel é um sistema web que utiliza a plataforma Redmine. É basicamente um softaware de gerenciamento e atividades.

O Fiscaliza possui 2 módulos para os fiscais utilizarem:
* Módulo de Teste ( Homologação ) : "https://sistemashm.anatel.gov.br/fiscaliza"
* Módulo de Produção: "https://sistemas.anatel.gov.br/fiscaliza/"

Ambos módulos precisam de um usuário e senha com acesso autorizado, em geral somente servidores da Anatel.

O Escopo desse módulo basicamente está encapsulado em 3 funções básicas:
* detalhar_inspecao - **Retorna as informações do estado atual da Issue ( Inspeção )**
* validar_dicionario - **Para dado dicionário de dados, formata-o para a API do Redmine**
* relatar_inspecao - **Atualiza e Issue com o dicionário de dados passado**

> Note: O escopo da terceira função `relatar_inspecao` possui escopo limitado. Atualmente somente é formatado e relatado Inspeções ( Issue ) to tipo "Uso do Espectro - Monitoração". Essa inspeção é a principal utilizada para as atividades de monitoração da Anatel e foi o que motivou a criação da presente biblioteca. Outras inspeções possuem campos distintos e assim exigem formatação distinta. Versões futuras poderão abarcar o relato de diferentes inspeções. 

In [ ]:
#hide
show_doc(detalhar_inspecao)

Vamos exemplificar a Inspeção de Teste `53646`

In [ ]:
login = getuser()
senha = getpass()
inspecao = '57683'

In [ ]:
detalhes = detalhar_inspecao(inspecao, login, senha)

In [ ]:
for k,v in detalhes.items():
    console.print(f'[red]{k} [blue]-> [green]{v}')

Agora vamos testar o retorno de informações para um Inspeção no Fiscaliza de produção `teste=False`.

In [ ]:
inspecao = '51804'
detalhes = detalhar_inspecao(inspecao, login, senha, teste=False)

In [ ]:
for k,v in detalhes.items():
    console.print(f'[blue]{k} [red]-> [green]{v}')

## Dicionário de Dados
A função anterior somente retorna as informações já constantes em dada Inspeção ( Issue ) do Fiscaliza. Para alterarmos ou atualizarmos dada inspeção, precisamos passar um dicionário de dados ou um caminho para um arquivo `.json` ou pickle `.pkl` onde conste esse dicionário de dados serializado. O exemplo seguinte mostra um dicionário de dados com as informações básicas constantes de uma monitoração e a formatação que elas são validadas:

In [ ]:
d = {}

d['Classe da Inspeção'] = 'Técnica' # str

d['Tipo de inspeção'] = 'Uso do Espectro - Monitoração' #str

d['Descrição da Inspeção'] = '''Atendimento da Denúncia AC202010213075425 (6104512), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, 
além da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.''' #str

d['Fiscal Responsável'] = 'Ronaldo da Silva Alves Batista' #str

d['Fiscais'] = ['Ronaldo da Silva Alves Batista', 'Paulo Diogo Costa', 'Mario Augusto Volpini'] #string ou lista de strings

# Windows
d['Html'] = 'D:\\OneDrive - ANATEL\\Monitoramento\\Processos\\53504.0005432021-55\\Guarulhos.html' #str

#ou 

#Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
d['Gerar Relatório'] = 1 # int 0 ou 1

d['Frequência Inicial']  = 54 #int ou float

d['Unidade da Frequência Inicial'] = 'MHz' #string

d['Frequência Final'] = 700 #int ou float

d['Unidade da Frequência Final'] = 'MHz' #string

d['Data de Início'] = '2021-03-19' #YYYY-MM-DD #string nesse formato

d['Data Limite'] = '2021-12-31'  #YYYY-MM-DD #string nesse formato

d['UF/Município'] = "SP/São Paulo" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

d['Serviços da Inspeção'] = ['230', '231', '800'] # String ou Lista de Strings

d['Qnt. de emissões na faixa'] = 12 # int

d['Emissões não autorizadas/desc'] = 70 # int

d['Horas de Preparação'] = 2 # int

d['Horas de Deslocamento'] = 0 # int

d['Horas de Execução'] = 32 # int

d['Horas de Conclusão'] = 6 # int

d['Latitude (coordenadas)'] = -22.94694 # float

d['Longitude (coordenadas)'] = -43.21944 # float

d['Uso de PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Ação de risco à vida criada?'] = 'Não' # string Sim | Não

d['Impossibilidade acesso online?'] = '0' # string '0' | '1'

d['Reservar Instrumentos?'] = '0' #string '0' = Não | '1' = 'Sim'

d['Utilizou algum instrumento?'] = '0'

#d['Reserva de Instrumentos'] = '' 

d['Notes'] = "Não foi constatada irregularidade no Período monitorado" # string

# No caso de uma tabela, string formatada como csv

d['Notes'] = """Faixa, Classe Especial, Classe A, Classe B, Classe C
                VHF-L,0,5,7,5
                VHF-H,0,12,1,0
                UHF,1,1,2,4
                FM,5,1,0,0
                RADCOM,0,0,0,0
                Outorgadas com indícios de irregularidades,1,2,3,4
            """

A API do Redmine possui formatos específicos de como esses campos devem ser submetidos, validar os formatos acima e fazer essa formatação exigida pela API do Redmine é o papel da função `validar_dicionario`.

In [ ]:
#hide
show_doc(validar_dicionario)

In [ ]:
inspecao = '57684'
dados = validar_dicionario('files/data.json', inspecao, login, senha)

In [ ]:
for k,v in dados.items():
    if k == "Html":
        v = str(v)
        v = v[:100] + '\n...\n' + v[-100:]
    console.print(f'[blue]{k} [red]-> [green]{v}')

## Relatar Inspeção
A função a seguir é a mais importante do módulo porque ela de fato altera os dados na plataforma Fiscaliza.

In [ ]:
#hide
show_doc(relatar_inspecao)

In [ ]:
inspecao = '57683'
relatar_inspecao(inspecao, login, senha, dados='files/dados.json', teste=True)

Usuário Autenticado com Sucesso 👍

Inspeção 57683 vinculada à Ação {'id_ACAO': 52876, 'nome_ACAO': 'ACAO_GR01_2021_0491', 
'descrição_ACAO': 'Teste'}

usage: ipykernel_launcher.py [-h] [--fiscaliza FISCALIZA] [--teste TESTE]
                             inspecao login senha
ipykernel_launcher.py: error: the following arguments are required: login, senha


SystemExit: 2

C:\Users\rsilva\Miniconda3\envs\fiscaliza\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


O dicionário de dados possui os dados completos obrigatórios para o relato, no entanto o relato é feito em algumas etapas, utilizando somente algumas das informações do dicionário:
* `Rascunho` para `Aguardando Execução`
* `Aguardando Execução` para `Em Andamento`
* `Em Andamento` para `Relatando`
* `Relatando` para `Relatada`

Na terceira etapa é gerado um relatório da monitoração no Sistema Eletrônico de Informações - SEI. Para executar a quarta etapa é preciso que esse relatório da terceira etapa esteja assinado. Por essa razão a função para na terceira etapa e informa da necessidade do relatório estar assinado.

Após o relatório ser assinado basta chamar a função com os mesmos argumentos para que a etapa final seja realizada.

In [ ]:
estado = relatar_inspecao(login, senha, inspecao, dados=dados, teste=True)